In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, DateType,TimestampType
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark import SQLContext
import json 
from datetime import datetime, timedelta
import math
import os
from IPython.core.display import display, HTML
from datetime import datetime
import pyspark.conf as conf
from IPython.display import clear_output

conf.autoBroadcastJoinThreshold = -1
display(HTML("<style>.container { width:100% !important; }</style>"))

from elasticsearch import Elasticsearch
import urllib3

urllib3.disable_warnings()

C:\Users\Maria\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
spark = SparkSession.builder.appName("TFG").getOrCreate()
elastic = Elasticsearch(hosts=['http://127.0.0.1:9200/'])

# SCHEMAS

In [3]:
schema_vehicle = StructType([ \
    StructField("DEVICE_ID",StringType(),True), \
    StructField("LATITUDE",DoubleType(),True), \
    StructField("LONGITUDE",DoubleType(),True), \
    StructField("DATETIME",TimestampType(),True), \
    StructField("SPEED",DoubleType(),True), \
  ])

# First look at the data

In [4]:
first_look_csv = 'data/fleet_original/logrecords_20210201_20210201_0_anon.csv' 

first_look_csv = spark.read.csv(first_look_csv, header=True, inferSchema=True, sep='|', schema=schema_vehicle)

first_look_csv.printSchema()
first_look_csv.show(3, truncate=False)

root
 |-- DEVICE_ID: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- DATETIME: timestamp (nullable = true)
 |-- SPEED: double (nullable = true)

+---------+----------+-----------+-----------------------+-----+
|DEVICE_ID|LATITUDE  |LONGITUDE  |DATETIME               |SPEED|
+---------+----------+-----------+-----------------------+-----+
|HQ3OZP1U |39.8995209|3.07118082 |2021-02-01 01:00:00.063|0.0  |
|GT42SJ4M |40.408371 |-3.66694403|2021-02-01 01:00:03    |42.0 |
|IILVL5RI |36.5030899|-6.26657295|2021-02-01 01:00:08.063|0.0  |
+---------+----------+-----------+-----------------------+-----+
only showing top 3 rows



# TRATAMIENTO DE DATOS

In [5]:
# rename columns
first_look_csv.withColumnRenamed('SPEED', 'SPEED (km/h)').show(3, truncate=False)

+---------+----------+-----------+-----------------------+------------+
|DEVICE_ID|LATITUDE  |LONGITUDE  |DATETIME               |SPEED (km/h)|
+---------+----------+-----------+-----------------------+------------+
|HQ3OZP1U |39.8995209|3.07118082 |2021-02-01 01:00:00.063|0.0         |
|GT42SJ4M |40.408371 |-3.66694403|2021-02-01 01:00:03    |42.0        |
|IILVL5RI |36.5030899|-6.26657295|2021-02-01 01:00:08.063|0.0         |
+---------+----------+-----------+-----------------------+------------+
only showing top 3 rows



# FUNCTIONS

In [6]:
def get_distance(lat1deg, lon1deg, lat2deg, lon2deg):
    '''
    return: distance in metres from 2 locations (lat, lon)
    '''
    if not all((lat1deg, lon1deg, lat2deg, lon2deg)):
        return 0.0
    
    #approximate radius of earth in m
    R = 6373000.0

    lat1 = math.radians(lat1deg)
    lon1 = math.radians(lon1deg)
    lat2 = math.radians(lat2deg)
    lon2 = math.radians(lon2deg)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

# register as a UDF 
get_distance_udf = F.udf(get_distance, DoubleType())

In [7]:
def get_delta_time(timestr): 
    '''
    return: deltatime as float in seconds from HH:MM:SS.mmm
    '''
    values = timestr.split(':')
    #Generate a timedelta
    delta = timedelta(hours=float(values[0]), minutes=float(values[1]), seconds=float(values[2]))
    #Represent in Seconds
    return delta.total_seconds()

# register as a UDF 
get_delta_time_udf = F.udf(get_delta_time, DoubleType())

In [8]:
def abs_time_delta(y,x): 
    '''
    return: duration in seconds
    '''
    if not all((x, y)):
        return 0.0
    delta = math.fabs((x-y).total_seconds())
    return delta

# register as a UDF 
abs_time_delta_udf = F.udf(abs_time_delta, DoubleType())

add dupe column

In [9]:
def remove_dupe_stationary(df_vehicles):
    '''
    create new column "dupe" to keep only one of the stationary vehicle entries
    '''
    w = Window.partitionBy("DEVICE_ID").orderBy(F.col("DATETIME"))
    
    df_vehicles = df_vehicles \
                .sort(F.col('DATETIME')) \
                .withColumn("dupe", \
                            F.when((F.col("SPEED") == 0) & \
                                   (F.lag("SPEED").over(w) == 0) & \
                                   (F.col("LONGITUDE") == F.lag("LONGITUDE").over(w)) & \
                                   (F.col("LATITUDE") == F.lag("LATITUDE").over(w)), 1).otherwise(0)) \
                .filter((F.col("dupe") == False) | (F.col("dupe").isNull())).drop("dupe")
    return df_vehicles

duration and distance

In [10]:
def add_duration_and_distance_columns(df_vehicles):
    '''
    calculate duration and distance of the route
    if distance is > 1km, it is considered a tunnel
    '''
    w = Window.partitionBy("DEVICE_ID").orderBy(F.col("DATETIME"))
    
    df_vehicles = df_vehicles.withColumn('PREV_LATITUDE', F.lag('LATITUDE').over(w)) \
                   .withColumn('PREV_LONGITUDE', F.lag('LONGITUDE').over(w)) \
                   .withColumn('PREV_DATETIME', F.lag('DATETIME').over(w)) \
                   .withColumn('DISTANCE', get_distance_udf(F.col('LATITUDE'), F.col('LONGITUDE'), F.col('PREV_LATITUDE'), F.col('PREV_LONGITUDE'))) \
                   .where(F.col('DISTANCE') != 0) \
                   .withColumn('DURATION', abs_time_delta_udf(F.col('PREV_DATETIME'), F.col('DATETIME'))) \
    .withColumn('TUNNEL', F.when(F.col('DISTANCE') > 1000, True).otherwise(False)) \
                   .withColumn('PREV_TUNNEL', F.lag('TUNNEL').over(w)) \
                   .drop('PREV_DATETIME')
                   #.withColumn('driving', F.when(F.col('SPEED') > 0, True).otherwise(False)) \
                   #.withColumn('prev_driving', F.lag('driving', 1, False).over(w)) \
                   
    return df_vehicles

total speed and distance

In [11]:
def get_speed_and_distance(df_vehicles, device_id):
    '''
    calculates the speed (km/h) and distance (km)
    '''
    
    df_duration_distance = df_vehicles.where(F.col('DEVICE_ID') == device_id).select('DURATION', 'DISTANCE')
    if df_duration_distance.count() == 0:
        return -1, -1
    df_sum = df_duration_distance.groupBy().sum()
    datasum = df_sum.collect()
    duration = datasum[0]['sum(DURATION)']
    distance = datasum[0]['sum(DISTANCE)']
    speed = 0.0
    
    if (duration is None) | (distance is None):
        speed = 0.0
        distance = 0.0
    else:
        speed = round(distance * 3.6 / duration, 2) # makes it km/h
        distance = round(distance / 1000.0, 2)
   
    df_duration_distance.unpersist()
    df_sum.unpersist()
    del df_duration_distance, df_sum, datasum
    return speed, distance


In [12]:
def get_driving_time(df_vehicles, device_id):
    '''
    calculates the total driving time (s)
    '''
    df_sum = df_vehicles.filter(F.col('DEVICE_ID') == device_id).select('DURATION').groupBy().sum('DURATION')

    driving_time = df_sum.collect()[0][0]
    
    return driving_time


In [13]:
def get_times(df_vehicles, device_id):
    '''
    get array with times of the vehicle in service. 
    The array are the hours vehicle is in service. For example: [8, 9, 10, 11, 12, 16, 17, 18] if the vehicle has been working 
    at 8:00 (really any point between 8:00 and 8:59), 9:00, 10:00, 11:00, 12:00, 16:00, 17:00 and 18:00
    '''

    timesArray = df_vehicles.filter(F.col('DEVICE_ID') == device_id).select('DATETIME').collect();
    times = []
    arr=[]
    arr = [False for i in range(24)] 
    for time in timesArray:
        arr[time['DATETIME'].hour] = True
    for idx, value in enumerate(arr):
        if value:
            times.append(idx)
    #print(arr, times)
    return times


vehicles geometry for geojson

In [14]:
def get_vehicle_geometry(df_block_vehicle):
    df_vehicle_geometry = df_block_vehicle.select('LONGITUDE', 'LATITUDE', 'PREV_LONGITUDE', 'PREV_LATITUDE', 'TUNNEL', 'PREV_TUNNEL') \
                                            .cache()
    data_vehicle = df_vehicle_geometry.rdd.toLocalIterator()
   
    df_vehicle_geometry.unpersist()
    return data_vehicle

# VEHICLES TREATMENT

In [15]:
def vehicles_treatment(df_vehicles):
    '''
    does all the treatment of the vehicles
    param df_vehicles
    return:
    '''
    devices_id = df_vehicles.select('DEVICE_ID').distinct().collect() 
    df_vehicles = remove_dupe_stationary(df_vehicles)
    df_vehicles = add_duration_and_distance_columns(df_vehicles)
    windowSpec = Window.partitionBy("DEVICE_ID").orderBy(F.col("DATETIME")).rowsBetween(Window.unboundedPreceding, Window.currentRow)
    # creates column SEGMENT_ID that states the line number
    # SEGMENT_ID is incremented when tunnel change of value (false->true or true->false)
    df_vehicles = df_vehicles.withColumn('SEGMENT_ID',F.count(F.when(F.col('TUNNEL') != F.col('PREV_TUNNEL'), F.lit(1))
                                                                                    ).over(windowSpec))
    
    df_vehicles = df_vehicles.cache()
    
    total_vehicles = len(devices_id)
    vehicle_number = 0
    
    for device_id in devices_id:
    
        vehicle_number = vehicle_number + 1
        #if vehicle_number ==10:
        #    break # maria quitar
        print("    Vehicle {} ({}/{}) {}".format(device_id.DEVICE_ID, vehicle_number, total_vehicles, datetime.now()))

        speed, distance = get_speed_and_distance(df_vehicles, device_id.DEVICE_ID)
        if distance <= 0:
            continue
        driving_time = get_driving_time(df_vehicles, device_id.DEVICE_ID)
        
        times = get_times(df_vehicles, device_id.DEVICE_ID)
        #print('      point A {}'.format(datetime.now()))
     
        vehicleSegmentIndex = 0
        segments = []
        while True:
            df_block_vehicle = df_vehicles.filter(F.col('DEVICE_ID') == device_id.DEVICE_ID).where(F.col('SEGMENT_ID') == vehicleSegmentIndex) \
                                .cache()
            
            vehicleSegmentIndex = vehicleSegmentIndex + 1
            headerVehicle = df_block_vehicle.head(1)

            if len(headerVehicle) == 0:
                break
        
            #print('      point B {}'.format(datetime.now()))
            df_vehicle_geometry = df_block_vehicle.select('LONGITUDE', 'LATITUDE', 'PREV_LONGITUDE', 'PREV_LATITUDE')
            data_geometry = df_vehicle_geometry.collect()
            df_vehicle_geometry.unpersist()
            #print('      point C {}'.format(datetime.now()))
        
            df_block_vehicle.unpersist()
            del df_block_vehicle

            #data_vehicle = get_vehicle_geometry(df_block_vehicle)
            #print('      point D {}'.format(datetime.now()))
            tunnel = True if headerVehicle[0]['TUNNEL'] is True else False
            date = headerVehicle[0]['DATETIME'].strftime("%Y/%m/%d")
            #print("date:", date)

            geometry = {"type": "LineString", "coordinates": []}
            for row in data_geometry:
                # if it is not the first segment, add the previous point
                if vehicleSegmentIndex > 1:
                    geometry["coordinates"].append([row['PREV_LONGITUDE'], row['PREV_LATITUDE']])
                geometry["coordinates"].append([row['LONGITUDE'], row['LATITUDE']])
            #print(geometry)
            if len(geometry["coordinates"]) > 0:
                #print("    Added {} with tunnel {} len={}".format(device_id.DEVICE_ID, tunnel, len(geometry["coordinates"])))
                segment = {'tunnel': tunnel,
                           'geometry': geometry};
                segments.append(segment);

            del data_geometry, geometry, df_vehicle_geometry
        
        #print('      point E {}'.format(datetime.now()))
 
        if len(segments) > 0:
            #print("    Added {} with tunnel {} len={}".format(device_id.DEVICE_ID, tunnel, len(geometry["coordinates"])))
            document={   'type': "Feature",
                         'device_id': device_id.DEVICE_ID,
                         'date': date, 
                         'speed': speed,
                         'distance': distance,
                         'driving_time': driving_time,
                         'service_times': times,
                         'segments': segments}
            elastic.index(index='fleet_index', document=document)
            #print(document)
   
    df_vehicles.unpersist()
    del devices_id, df_vehicles, windowSpec

read all vehicles

In [16]:
def get_vehicles_initial(file):
    df_vehicles_initial = spark.read.csv(file, header=True, inferSchema=True, sep='|',schema=schema_vehicle)
    return df_vehicles_initial

In [17]:
MAIN_START_TIME = datetime.now()
starting_time = datetime.now()
directory = "data/fleet_original/"
for filename in os.listdir(directory):
    # leemos csv
    if filename.endswith(".csv"): 

        print(directory + filename)
        dia = filename.split('_')[1][6:]
        mes = filename.split('_')[1][4:6]
        ano = filename.split('_')[1][:4]
        
        alreadyDoneDays = 23
        daysToDo = 8
        if int(dia) <= alreadyDoneDays: # skip already done days 
            continue
        if int(dia) > (alreadyDoneDays+daysToDo):  
            continue
            
        df_vehicles = get_vehicles_initial(directory + filename)
        print('df_vehicle loaded at {}'.format(datetime.now()))

        vehicles_treatment(df_vehicles)

        df_vehicles.unpersist()
        #del df_vehicles
        #clear_output(wait=True)

        print('save vehicles from {} done at {}'.format(filename, datetime.now()))
        
        
print('vehicles_treatment process finished (duration = {} hours, {} minutes)'.format(
    ((datetime.now() - MAIN_START_TIME).seconds)//3600,
    (((datetime.now() - MAIN_START_TIME).seconds)//60)%60))

data/fleet_original/logrecords_20210201_20210201_0_anon.csv
data/fleet_original/logrecords_20210202_20210202_0_anon.csv
data/fleet_original/logrecords_20210203_20210203_0_anon.csv
data/fleet_original/logrecords_20210204_20210204_0_anon.csv
data/fleet_original/logrecords_20210205_20210205_0_anon.csv
data/fleet_original/logrecords_20210206_20210206_0_anon.csv
data/fleet_original/logrecords_20210207_20210207_0_anon.csv
data/fleet_original/logrecords_20210208_20210208_0_anon.csv
data/fleet_original/logrecords_20210209_20210209_0_anon.csv
data/fleet_original/logrecords_20210210_20210210_0_anon.csv
data/fleet_original/logrecords_20210211_20210211_0_anon.csv
data/fleet_original/logrecords_20210212_20210212_0_anon.csv
data/fleet_original/logrecords_20210213_20210213_0_anon.csv
data/fleet_original/logrecords_20210214_20210214_0_anon.csv
data/fleet_original/logrecords_20210215_20210215_0_anon.csv
data/fleet_original/logrecords_20210216_20210216_0_anon.csv
data/fleet_original/logrecords_20210217_

    Vehicle ZY6N8EFL (117/1275) 2022-05-07 11:08:47.172058
    Vehicle FAJI7V2I (118/1275) 2022-05-07 11:08:50.878066
    Vehicle TE8RDGRT (119/1275) 2022-05-07 11:08:51.610690
    Vehicle ONM3VK28 (120/1275) 2022-05-07 11:08:55.110433
    Vehicle WX0UYMFQ (121/1275) 2022-05-07 11:08:55.908654
    Vehicle 00H12ZKQ (122/1275) 2022-05-07 11:09:02.358240
    Vehicle TNP7VPOZ (123/1275) 2022-05-07 11:09:11.696190
    Vehicle F7NO3YCA (124/1275) 2022-05-07 11:09:15.147012
    Vehicle BM99JO54 (125/1275) 2022-05-07 11:09:27.410309
    Vehicle OR5AXHOD (126/1275) 2022-05-07 11:09:39.777328
    Vehicle XDCV0T16 (127/1275) 2022-05-07 11:09:54.612658
    Vehicle D9V67XDU (128/1275) 2022-05-07 11:09:58.844493
    Vehicle BR3TP256 (129/1275) 2022-05-07 11:10:05.869122
    Vehicle M7SX11QK (130/1275) 2022-05-07 11:10:19.672269
    Vehicle 5UAE0GI8 (131/1275) 2022-05-07 11:10:20.387274
    Vehicle NEENXEPP (132/1275) 2022-05-07 11:10:29.544696
    Vehicle NYJCQRRD (133/1275) 2022-05-07 11:10:32.7555

    Vehicle I205CLF3 (256/1275) 2022-05-07 11:26:37.888847
    Vehicle YXJVBZEJ (257/1275) 2022-05-07 11:26:43.083740
    Vehicle GCDALDSO (258/1275) 2022-05-07 11:27:14.839980
    Vehicle TGQL10Y0 (259/1275) 2022-05-07 11:27:19.131042
    Vehicle MH8CPNEQ (260/1275) 2022-05-07 11:27:22.546414
    Vehicle UV618PPC (261/1275) 2022-05-07 11:27:44.646875
    Vehicle MOLKW4PF (262/1275) 2022-05-07 11:27:48.652579
    Vehicle ZZAJ8BTV (263/1275) 2022-05-07 11:27:54.943107
    Vehicle ZY8CI7HI (264/1275) 2022-05-07 11:27:55.583242
    Vehicle 8IY0BP1G (265/1275) 2022-05-07 11:27:56.287012
    Vehicle ASKQB67I (266/1275) 2022-05-07 11:27:56.987259
    Vehicle 54B53X2F (267/1275) 2022-05-07 11:27:57.823824
    Vehicle 94N3CTLK (268/1275) 2022-05-07 11:28:14.633564
    Vehicle 00LXDZCW (269/1275) 2022-05-07 11:28:22.707068
    Vehicle 2QVJIO7G (270/1275) 2022-05-07 11:28:26.720709
    Vehicle 35WO91X5 (271/1275) 2022-05-07 11:28:38.244724
    Vehicle 7CH5D8IF (272/1275) 2022-05-07 11:28:41.6568

    Vehicle A6P4CMBW (395/1275) 2022-05-07 11:39:24.373681
    Vehicle DYKKM0IJ (396/1275) 2022-05-07 11:39:29.839872
    Vehicle M70ER0HY (397/1275) 2022-05-07 11:39:33.207671
    Vehicle HIZYB1NV (398/1275) 2022-05-07 11:39:37.705586
    Vehicle GTX8TUTH (399/1275) 2022-05-07 11:39:49.657522
    Vehicle M5GIYHXK (400/1275) 2022-05-07 11:40:32.215309
    Vehicle ODTWP7DK (401/1275) 2022-05-07 11:40:36.193313
    Vehicle FU4KZLDP (402/1275) 2022-05-07 11:40:41.948255
    Vehicle T58JM2VW (403/1275) 2022-05-07 11:40:46.347944
    Vehicle 8UCRK7SD (404/1275) 2022-05-07 11:40:54.083847
    Vehicle 45QJGGUG (405/1275) 2022-05-07 11:40:57.456697
    Vehicle 9NHPMZVM (406/1275) 2022-05-07 11:41:19.019006
    Vehicle 6RJGQBXZ (407/1275) 2022-05-07 11:41:19.666055
    Vehicle AE6XFOFT (408/1275) 2022-05-07 11:41:20.320310
    Vehicle URV0HDMZ (409/1275) 2022-05-07 11:41:21.009993
    Vehicle I3KCLR75 (410/1275) 2022-05-07 11:41:24.928419
    Vehicle ZYIM5AR4 (411/1275) 2022-05-07 11:41:30.8576

    Vehicle RMMGE252 (534/1275) 2022-05-07 11:55:47.912911
    Vehicle HE6O8RO3 (535/1275) 2022-05-07 11:55:52.670794
    Vehicle UFP86HFE (536/1275) 2022-05-07 11:55:55.941795
    Vehicle JJ7C6Z77 (537/1275) 2022-05-07 11:55:56.591520
    Vehicle 7LI2TGG8 (538/1275) 2022-05-07 11:55:59.876636
    Vehicle 3STDHEOB (539/1275) 2022-05-07 11:56:03.138399
    Vehicle 2HP8H3UR (540/1275) 2022-05-07 11:56:16.698945
    Vehicle QAV6UU4A (541/1275) 2022-05-07 11:56:21.054393
    Vehicle EYN2DYWC (542/1275) 2022-05-07 11:56:28.105610
    Vehicle Y4ERYKI5 (543/1275) 2022-05-07 11:56:38.551296
    Vehicle YGFES4OC (544/1275) 2022-05-07 11:56:41.792475
    Vehicle GIMQ11HF (545/1275) 2022-05-07 11:56:44.923445
    Vehicle JD655SOM (546/1275) 2022-05-07 11:56:48.429038
    Vehicle G6TLEL8Y (547/1275) 2022-05-07 11:56:53.945901
    Vehicle P4NC962X (548/1275) 2022-05-07 11:57:20.618902
    Vehicle HA4WNKQA (549/1275) 2022-05-07 11:57:21.260337
    Vehicle T7US804A (550/1275) 2022-05-07 11:57:46.9787

...

    Vehicle QXW1Q5CY (626/1217) 2022-05-07 22:16:10.851441
    Vehicle 053O3FYE (627/1217) 2022-05-07 22:16:12.209157
    Vehicle 7DG01VSL (628/1217) 2022-05-07 22:16:13.684112
    Vehicle DBP5BEAX (629/1217) 2022-05-07 22:16:14.929326
    Vehicle 78HPX6YI (630/1217) 2022-05-07 22:16:16.857355
    Vehicle Q8YQCK5O (631/1217) 2022-05-07 22:16:18.239394
    Vehicle JB6US3F9 (632/1217) 2022-05-07 22:16:20.465634
    Vehicle F1WDWDHW (633/1217) 2022-05-07 22:16:21.593657
    Vehicle VYILGFJM (634/1217) 2022-05-07 22:16:22.749020
    Vehicle Q562NOE3 (635/1217) 2022-05-07 22:16:24.085429
    Vehicle LEFR16O4 (636/1217) 2022-05-07 22:16:25.233621
    Vehicle OMG3GO3Y (637/1217) 2022-05-07 22:16:27.297250
    Vehicle 1O1F2O5X (638/1217) 2022-05-07 22:16:35.299695
    Vehicle VFXOKJ9T (639/1217) 2022-05-07 22:16:36.398113
    Vehicle QY9FS8Z6 (640/1217) 2022-05-07 22:16:37.582446
    Vehicle 7SW37NLA (641/1217) 2022-05-07 22:16:38.907010
    Vehicle C79QENFX (642/1217) 2022-05-07 22:16:39.9745

    Vehicle G2JELJ0F (765/1217) 2022-05-07 22:19:44.291952
    Vehicle CMK865KA (766/1217) 2022-05-07 22:19:45.557391
    Vehicle S5KVQMB4 (767/1217) 2022-05-07 22:19:47.574802
    Vehicle DJAZ3PO7 (768/1217) 2022-05-07 22:19:48.864584
    Vehicle QBNNOV1O (769/1217) 2022-05-07 22:19:50.918819
    Vehicle 1WHP3QMW (770/1217) 2022-05-07 22:19:51.994851
    Vehicle 0MCOS47D (771/1217) 2022-05-07 22:19:53.991692
    Vehicle TRBM4G71 (772/1217) 2022-05-07 22:19:55.085605
    Vehicle DW9DDBKL (773/1217) 2022-05-07 22:19:56.218514
    Vehicle B0E6U0OB (774/1217) 2022-05-07 22:19:58.065688
    Vehicle 7ALYFLFA (775/1217) 2022-05-07 22:19:59.063468
    Vehicle ZQ1EN8JZ (776/1217) 2022-05-07 22:20:00.186367
    Vehicle KEOMNM0H (777/1217) 2022-05-07 22:20:01.622006
    Vehicle 32XWTSYD (778/1217) 2022-05-07 22:20:02.949203
    Vehicle H6WS111D (779/1217) 2022-05-07 22:20:04.110213
    Vehicle RBAYAL5U (780/1217) 2022-05-07 22:20:05.540605
    Vehicle 65E24T4E (781/1217) 2022-05-07 22:20:06.9954

    Vehicle 7SSQLQPB (904/1217) 2022-05-07 22:23:51.853089
    Vehicle 8Z2NOAJS (905/1217) 2022-05-07 22:23:53.215656
    Vehicle I2LWQYZ5 (906/1217) 2022-05-07 22:23:54.446453
    Vehicle LMB1LQAB (907/1217) 2022-05-07 22:23:55.727948
    Vehicle IYXREWSN (908/1217) 2022-05-07 22:23:57.036195
    Vehicle ZAZKDKAH (909/1217) 2022-05-07 22:23:58.361253
    Vehicle ZE8794WX (910/1217) 2022-05-07 22:23:59.708557
    Vehicle KUD50MVN (911/1217) 2022-05-07 22:24:00.993078
    Vehicle Z1WKA9YI (912/1217) 2022-05-07 22:24:03.030993
    Vehicle WWOESGW9 (913/1217) 2022-05-07 22:24:04.127561
    Vehicle O8OFYDS3 (914/1217) 2022-05-07 22:24:05.625875
    Vehicle W0EMKMY4 (915/1217) 2022-05-07 22:24:06.824747
    Vehicle RJNY4CLU (916/1217) 2022-05-07 22:24:07.950530
    Vehicle XJQSJEFR (917/1217) 2022-05-07 22:24:09.128963
    Vehicle NDRZL1OP (918/1217) 2022-05-07 22:24:10.311826
    Vehicle 6QNVZKEO (919/1217) 2022-05-07 22:24:12.234741
    Vehicle Z4LEDMZX (920/1217) 2022-05-07 22:24:13.5066

    Vehicle WS3V5MZH (1043/1217) 2022-05-07 22:27:19.363923
    Vehicle DHIMKX0W (1044/1217) 2022-05-07 22:27:21.257161
    Vehicle ECQDH33B (1045/1217) 2022-05-07 22:27:22.390856
    Vehicle 3BS5BGKI (1046/1217) 2022-05-07 22:27:24.321027
    Vehicle B2Y25LS2 (1047/1217) 2022-05-07 22:27:25.393941
    Vehicle 8JTJQODJ (1048/1217) 2022-05-07 22:27:26.827881
    Vehicle S8TQDI8W (1049/1217) 2022-05-07 22:27:27.919980
    Vehicle DOL1ZQIN (1050/1217) 2022-05-07 22:27:29.141269
    Vehicle 4BR73AZB (1051/1217) 2022-05-07 22:27:30.287390
    Vehicle ZCWEPSIF (1052/1217) 2022-05-07 22:27:31.490706
    Vehicle 6KK4NP0W (1053/1217) 2022-05-07 22:27:32.914772
    Vehicle IBPK0AZU (1054/1217) 2022-05-07 22:27:34.642581
    Vehicle UFLRMMY5 (1055/1217) 2022-05-07 22:27:36.512688
    Vehicle XA7WUGC9 (1056/1217) 2022-05-07 22:27:37.717890
    Vehicle M41S05OE (1057/1217) 2022-05-07 22:27:38.839314
    Vehicle RLS0GKXQ (1058/1217) 2022-05-07 22:27:41.429660
    Vehicle Q169JF21 (1059/1217) 2022-05

    Vehicle 50EIZG1L (1180/1217) 2022-05-07 22:30:36.216959
    Vehicle S6HI4SJJ (1181/1217) 2022-05-07 22:30:38.502286
    Vehicle TLZD1KJC (1182/1217) 2022-05-07 22:30:39.651121
    Vehicle NWHZ216Z (1183/1217) 2022-05-07 22:30:41.134999
    Vehicle KW1R6DJS (1184/1217) 2022-05-07 22:30:43.109481
    Vehicle Y7DW1GAU (1185/1217) 2022-05-07 22:30:44.337036
    Vehicle 4RLRZRYM (1186/1217) 2022-05-07 22:30:45.758731
    Vehicle 0QCY118Z (1187/1217) 2022-05-07 22:30:47.043829
    Vehicle QIZ7W9VK (1188/1217) 2022-05-07 22:30:48.202875
    Vehicle DA9Q6QF9 (1189/1217) 2022-05-07 22:30:49.624504
    Vehicle LQ9BMDBV (1190/1217) 2022-05-07 22:30:50.747076
    Vehicle QFEBMW3Z (1191/1217) 2022-05-07 22:30:52.543584
    Vehicle J00GH3UG (1192/1217) 2022-05-07 22:30:54.255452
    Vehicle 2MF9MIS2 (1193/1217) 2022-05-07 22:30:55.361879
    Vehicle 7DZH19WM (1194/1217) 2022-05-07 22:30:56.621524
    Vehicle 7DH6D8KG (1195/1217) 2022-05-07 22:30:58.453527
    Vehicle 081KQY4G (1196/1217) 2022-05

NameError: name 'f' is not defined